# Visorando road map inference from GPS trajectory using UNet

## Inputs import and configuration
- **Training configuration**:
    - Choose number of epochs and batch size.
    - Choose network architecture and loss function.
    - Choose number of units for the first layer.

- **Data configuration**:
    - Choose your training datasets assuming they have been created with _createDataset.py_ and _createTruth.py_.
    - Choose your training resolution assuming they exist.
    - Choose your channels. Comment the ones you don't want to train on.
    - Choose image size.
    - Choose if you want augmentation.


In [ ]:
from imageTiling import preprocessTrainData, preprocessTestData, loadPatches
from nn import trainModel, loadModel, doPredictions
from definitions import loadRaster, convertResultToRaster
from converter import resultToGeojson
from evaluator import main_eval

confTraining = {
    'epoch' : 20,
    'batch_size' : 8,
    'network' : 'UNet',
    'units': 2,
    'loss' : 'dice_loss',
}

confData = {
    'datasets' : [
        'labaroche',
        'hunawihr',
        #'linthal',
        #'blaesheim',
        #'haguenau',
    ],
    'resolutions' : [
        2.2,
        2.7,
        3.1
    ],
    'channels' : [
        'binary',
        'heatmap',
        'distance',
        'speed',
        'acceleration',
        'altitude',
        'slope',
        'bearing1',
        'bearing2',
        'bearing3',
        'bearing4',
        'bearing5',
        'bearing6',
        'bearing7',
        'bearing8',
        'bearing_deviation',
        'bearing_difference',
    ],
    'image_size' : 208,
    'augment' : True,
}

## Training
### Convert data into patches

In [ ]:
preprocessTrainData(confData, output_dir='train_examples')

### Train the model and save it

In [ ]:
model, _ = trainModel(confTraining, confData, train_directory='train_examples')
model.save('path')

## Do Predictions
1) First step is to choose the dataset/resolution you want to predict

In [ ]:
dataset = 'ribeauville'
resolution = 2.7
raster = loadRaster(dataset, resolution)

2) Load the model you want to predict with

In [ ]:
model = loadModel('path', confTraining)

3) Define a **pas** and a **pad**.

For prediction, we don't tile the raster with a fixed patch size of (X, X). Instead, we tile it with a higher frequency to enable multiple predictions of same zones. The parameter **pas** is used to define the spacing between each patch, and **pad** is used when merging the predictions that overlap to retain only the patch centers.

In [ ]:
pas = confData['image_size'] / 2
pad = 20

4) Preprocess the test data

This create the test data. Here you can specify if you want to permute a channel by adding a value to the parameter **channelToPermute**

In [ ]:
preprocessTestData(raster, confData, pas, output_directory='test_examples')

5) Generate result in raster and Geojson

In [ ]:
patchs = loadPatches(confData, directory='test_examples')
result = doPredictions(patchs, model, confData, pas, pad=20)
result_raster = convertResultToRaster(result, raster)
result_geojson = resultToGeojson(result_raster, levels=[0.15,1])

## Evaluation

Here is your evaluation process. It evaluate intersections, segments and connectivity 

In [ ]:
prediction_path='result.geojson'
truth_path='truthhome.geojson'

FINT, FTOP, FITO = main_eval(prediction_path, truth_path)
print(FINT, FTOP, FITO)